# Transfer setup from one account to another

DOES NOT CURRENTLY SUPPORT CHANNELS

In [ ]:
from bwproject import BWProject
from bwresources import BWQueries, BWGroups, BWAuthorLists, BWSiteLists, BWLocationLists, BWTags, BWCategories, BWRules, BWSignals
from pprint import pprint
import logging
import requests
import datetime

In [ ]:
logger = logging.getLogger("bwapi")
#can also set to .DEBUG, .WARN, .ERROR or .CRITICAL (in descending order)
logger.setLevel(logging.INFO)

Set this information before starting your script

In [ ]:
origin_acct = "[account]"
origin_pass = "[password]"
origin_proj = "[project]"

destination_acct = "[account]"
destination_pass = "[password]"
destination_proj = "[project]"

In [ ]:
backfill_date = "2015-06-01"

Leave these lists blank if you want to transfer everything

In [ ]:
queries_to_transfer = []
rules_to_transfer = []

# Starting Script

In [ ]:
origin = BWProject(username = origin_acct, 
                   password = origin_pass, 
                   project = origin_proj,
                   grant_type = "partner-password",
                   client_id = "partner-api-client")

destination = BWProject(username = destination_acct, 
                        password = destination_pass, 
                        project = destination_proj,
                        grant_type = "partner-password",
                        client_id = "partner-api-client")

## Queries

In [ ]:
origin_queries = BWQueries(origin)
destination_queries = BWQueries(destination)

In [ ]:
query_list = []
channel_list = []
found = []
not_uploaded = []


for query in origin_queries.ids:
    if (not queries_to_transfer) or (query in queries_to_transfer):
        found.append(query)
        query_info = origin_queries.get(query)

        if query_info["type"] == "search string":
            upload_info = {k: query_info[k] for k in ("name", 
                                                 "includedTerms", 
                                                 "languages", 
                                                 "lockedByUsername", 
                                                 "lockedQuery", 
                                                 "samplePercent")}
            query_list.append(upload_info)

        elif query_info["type"] == "twitter":
            upload_info = {"name": query_info["name"],
                          "handle": query_info["twitterScreenName"],
                          "channel_type": "twitter"}
            channel_list.append(upload_info)

        #we don't currently support facebook channels
        else:
            not_uploaded.append(query_info["name"])


In [ ]:
for query in query_list:
    print("Uploading: " + query["name"])
    destination_queries.upload(**query)

In [ ]:
invalid_channels = []
for channel in channel_list:
    response = origin.bare_request(verb=requests.get,
                                    address_root="http://app.brandwatch.net/",
                                    address_suffix="twitterapi/users/show.json",
                                    params={"screen_name": channel["handle"]})
    if "id_str" not in response:
        print(channel["name"])
        print(response)
        invalid_channels.append(channel["name"])

In [ ]:
for channel in channel_list:
    if channel["name"] not in invalid_channels:
        print("Uploading: " + channel["name"])
        destination_queries.upload_channel(**channel)

In [ ]:
if not_uploaded:
    print("Could not upload the following queries:")
    print(not_uploaded)
    
not_found = set(queries_to_transfer) - set(found)
if not_found:
    print("Could not find the following queries:")
    print(not_found)
    
if invalid_channels:
    print("The following channels are no longer valid and could not be transferred:")
    print(invalid_channels)
    
if queries_to_transfer:
    print("The following queries were not transferred:")
    print(set(queries_to_transfer) - set(destination_queries.ids))

## Groups

In [ ]:
origin_groups = BWGroups(origin)
destination_groups = BWGroups(destination)

In [ ]:
groups_not_transferred = []

for group in origin_groups.ids:
    group_info = origin_groups.get(group)
    upload_info = {}
    upload_info["name"] = group_info["name"]
    upload_info["queries"] = []
    for query in group_info["queries"]:
        if query["name"] in list(destination_queries.ids):
            upload_info["queries"].append(query["name"])
    if upload_info["queries"]:
        destination_groups.upload(**upload_info)
    else:
        groups_not_transferred.append(upload_info["name"])

if groups_not_transferred:
    print("The following groups will not be transferred because the queries they were attached to were not transferred:")
    print(groups_not_transferred)

## Author Lists

In [ ]:
origin_authorlists = BWAuthorLists(origin)
destination_authorlists = BWAuthorLists(destination)

In [ ]:
for authorlist in origin_authorlists.ids:
    destination_authorlists.upload(name = authorlist,
                                   authors = origin_authorlists.get(authorlist)["authors"])

## Site Lists

In [ ]:
origin_sitelists = BWSiteLists(origin)
destination_sitelists = BWSiteLists(destination)

In [ ]:
for sitelist in origin_sitelists.ids:
    destination_sitelists.upload(name = sitelist,
                                   domains = origin_sitelists.get(sitelist)["domains"])

## Location Lists

In [ ]:
origin_locationlists = BWLocationLists(origin)
destination_locationlists = BWLocationLists(destination)

In [ ]:
for locationlist in origin_locationlists.ids:
    destination_locationlists.upload(name = locationlist,
                                   locations = origin_locationlists.get(locationlist)["locations"])

## Rules

In [ ]:
origin_rules = BWRules(origin)
destination_rules = BWRules(destination)

In [ ]:
rules = origin_rules.get()

In [ ]:
new_rules = []
not_transferred_query_deleted = []
not_transferred_queries_not_in_acct = []
not_transferred_no_action = []
for rule in rules:
    if (not rules_to_transfer) or (rule["name"] in rules_to_transfer):
        if ("action" not in rule["ruleAction"]) or ("setting" not in rule["ruleAction"]):
            not_transferred_no_action.append(rule["name"])
        else:
            rule["ruleAction"] = destination_rules.rule_action(**rule["ruleAction"])

            if (rule["filter"]["queryName"] == "Whole Project"):
                rule["filter"].pop("queryName", None)
                rule["filter"] = destination_rules.filters(**rule["filter"])
                new_rules.append(rule)
            elif rule["filter"]["queryName"] == "":
                not_transferred_query_deleted.append(rule["name"])
                pass
            else:
                queries_for_new_rule = []
                for query in rule["filter"]["queryName"]:
                    if query in destination_queries.ids:
                        queries_for_new_rule.append(query)
                if queries_for_new_rule:
                    rule["filter"]["queryName"] = queries_for_new_rule
                    rule["filter"] = destination_rules.filters(**rule["filter"])
                    new_rules.append(rule)
                else:
                    not_transferred_queries_not_in_acct.append(rule["name"])

if not_transferred_query_deleted:           
    print("The following rules will not be transferred because they were not attached to any queries in the original acct:")
    print(not_transferred_query_deleted)

if not_transferred_queries_not_in_acct:
    print("The following rules will not be transferred because the queries they were attached to were not transferred:")
    print(not_transferred_queries_not_in_acct)   
    
if not_transferred_no_action:
    print("The following rules will not be transferred because they have no action in the origin account:")
    print(not_transferred_no_action) 

In [ ]:
for rule in new_rules:
    print("Uploading: "+ rule["name"])
    destination_rules.upload(**rule)

## Signals

In [ ]:
origin_signals = BWSignals(origin)
destination_signals = BWSignals(destination)

In [ ]:
signals = origin_signals.get()

In [ ]:
origin_categories = BWCategories(origin)
origin_tags = BWTags(origin)

In [ ]:
inv_queries = {v: k for k, v in origin_queries.ids.items()}
inv_tags = {v: k for k, v in origin_tags.ids.items()}

inv_categories = {}
for category in origin_categories.ids:
    for child in origin_categories.ids[category]["children"]:
        inv_categories[origin_categories.ids[category]["children"][child]] = child

In [ ]:
signals_to_upload = []
for signal in signals:
    if signal["projectId"] == origin.project_id:
        formatting = {"name": signal["name"],
                      "queries": [],
                      "category": [],
                      "xcategory": [],
                      "tags": [],
                      "xtags": [],
                      "subscribers": signal["subscribers"]}
        for category in signal["includeCategoryIds"]:
            if category not in inv_categories:
                pprint("Couldn't find "+ str(category) + "for signal " +signal["name"])
            else:
                formatting["category"].append(inv_categories[category])
        for category in signal["excludeCategoryIds"]:
            if category not in inv_categories:
                pprint("Couldn't find "+ str(category) + "for signal " +signal["name"])
            else:
                formatting["xcategory"].append(inv_categories[category])
                
        for tag in signal["includeTagIds"]:
            if tag not in inv_tags:
                pprint("Couldn't find "+ str(tag) + "for signal " +signal["name"])
            else:
                formatting["tags"].append(inv_tags[tag])
        for tag in signal["excludeTagIds"]:
            if tag not in inv_tags:
                pprint("Couldn't find "+ str(tag) + "for signal " +signal["name"])
            else:
                formatting["xtags"].append(inv_tags[tag])
                
                
        for query in signal["queryIds"]:
            if query not in inv_queries:
                pprint("Couldn't find "+ str(query) + "for signal " +signal["name"])
            else:
                formatting["queries"].append(inv_queries[query])
                signals_to_upload.append(formatting)

In [ ]:
destination_signals.upload_all(signals_to_upload)

## Backfill

In [ ]:
for query in destination_queries.ids:
    destination_queries.backfill(destination_queries.ids[query], backfill_date)